In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import time 
import string
import os
import nltk
#from nltk.stem import WordNetLemmatize
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from warnings import filterwarnings

In [2]:
# Assessing CHROME and opening page
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
driver.get("https://www.google.com")

browser = driver
url = "https://www.google.com/maps/place/Christ+(Deemed+to+be+University)/@12.9364929,77.6025247,17z/data=!4m7!3m6!1s0x3bae15b277a93807:0x88518f37b39dabd0!8m2!3d12.9362362!4d77.6061888!9m1!1b1"
browser.get(url)
response = BeautifulSoup(driver.page_source, 'html.parser')

time.sleep(1)

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 38.6MB/s]


In [3]:
# scrolling to the end of the Reviews webpage on chrome to load all avaliable reviews

i = 0

SCROLL_PAUSE_TIME = 10

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

number = 0

while True:
    number = number+1

    # Scroll down to bottom

    ele = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)

    # Wait to load page

    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    print(f'last height: {last_height}')

    ele = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

    print(f'new height: {new_height}')

    if number == 5:
        break

    if new_height == last_height:
        break

    print('cont')
    last_height = new_height


last height: 556
new height: 49
cont
last height: 49
new height: 48
cont
last height: 48
new height: 48


In [20]:
# accessing and extracting the elements from each review on the webpage 

reviews = []  
time_ago = []
ratings = []
names = []
    
review = driver.find_elements(By.CLASS_NAME,"wiI7pd")

for j in review:
    p = str(j.text)
    if p is not None: 
        reviews.append(p)
    else:
        s = 'no review'
        reviews.append(s)
    
    
time = driver.find_elements(By.CLASS_NAME,"rsqaWe")

for k in time:
    o = str(k.text)
    time_ago.append(o)
    

rating = driver.find_elements(By.CLASS_NAME,'kvMYJc')

for l in rating:
    c = str((l.get_attribute("aria-label")))
    ratings.append(c)

username = driver.find_elements(By.CLASS_NAME,"d4r55")   

for m in username:
    s = str(m.text)
    names.append(s)
       


In [21]:
#finding length
print(len(reviews))
print(len(time_ago))
print(len(ratings))
print(len(names))

1148
1140
1140
1140


In [22]:
reviews = pd.DataFrame(reviews)
norev = reviews[ (reviews == 'no review') ].index
norev.drop(norev)
#names to Dataframe
names = pd.DataFrame(names)
len(norev)

1148

In [18]:
#splitting the ratings to extract only the numbers ie " 5 stars" -> '5'

split_rate =[]
for r in range (len(ratings)):
    b = ratings[r]
    a = (b[1])
    split_rate.append(a)

split = pd.DataFrame(split_rate)
len(split)

1140

In [19]:
# creating data frame with scraped values 
data=names
data['USERNAME'] = names
data['REVIEWS'] = reviews
#data['RATINGS'] = split_rate
data['RATINGS'] = split
data['TIME AGO'] = time_ago

data = data[['USERNAME','REVIEWS','RATINGS','TIME AGO']]
data.head(10)

ValueError: Columns must be same length as key

In [9]:
# creating csv file with dataframe
#data.to_csv('Google_reviews_central.csv')

In [10]:
# loading created csv file
df = pd.read_csv("Google_reviews_central.csv")
df.drop(columns = df.columns[0], axis = 1, inplace= True)
df.head(5)

,USERNAME,REVIEWS,RATINGS,TIME AGO
0,ARPIT SOOD,Recently went to Christ University for a Marke...,,4 months ago
1,Avik,This is a beautiful campus. There are many tre...,,a month ago
2,Sonakshi Massey,I was a part of the BBA program offered by the...,,a day ago
3,jino narzary,Beautiful campus and really lovely atmosphere ...,,a year ago
4,Senthilkumar N K,"Christ University offers UG, PG, and Ph.D. pro...",,8 months ago


In [11]:
# cleaning dataset
#Drop rows with missing values
df.dropna(inplace=True)
# Assign all the rating >3 as 1 else 0
df=df[df['RATINGS']!=3]
df['POSITIVELY RATED']= np.where(df['RATINGS']>3,1,0)
df

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
#cleaning the data
clean = df.REVIEWS.values.tolist()
import re
clean=[re.sub(r"@\S+", "",sent) for sent in clean]
# Removing URL
clean=[re.sub("http[s]?\://\S+","",sent) for sent in clean]
#Removing hashtag (#…)
clean=[re.sub(r"#\S+", "",sent) for sent in clean]# removing #NLP
# Removing Numbers 
clean=[re.sub(r"[0-9]", "",sent) for sent in clean]
#Removing text in brackets ([…] or (…))
clean=[re.sub(r"(\(.*\))|(\[.*\])", "",sent) for sent in clean] 
# Removing line or tab character (\n, \r, \t..)
clean=[re.sub(r"\n", "",sent) for sent in clean] # removing \n
# Removing Punctuations
clean= [re.sub(r'[^\w\s]', '',sent) for sent in clean]
clean=[sent.lower() for sent in clean]

In [ ]:
# lemmatizing the contents to bring the words to their base form
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
lemmatizer = nltk.stem.WordNetLemmatizer()
post_lemma = []
for i in range (len(clean)):
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(clean[i])
    # Lemmatize list of words and join
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    post_lemma.append(lemmatized_output)
    
post_lemma = pd.DataFrame(post_lemma)
df['LEMMATIZED'] = post_lemma 
df = df.dropna()

In [ ]:
# USING NLTK TO REMOVE STOP WORDS
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['christ university','university','universities','bengaluru',
                   'bangalore','hosur','rd','road','christite','christmas','decorations'
                  'campus','education','knowledge','college','colleges','placements','ranked','deemed','central','block',
                  'christ'])

df['CLEAN REVIEWS'] = df['LEMMATIZED'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [ ]:
df.head(10)

# VADER - NON-ML TECHINQUES (LEXICON BASED)

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #initiating VADER instance
import nltk
#nltk.download('vader_lexicon') 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [ ]:
#Calculating score for each tweet in the dataframe/dataset

i=0 

compval1 = [ ]  

while i<len(df):
    
    k = analyser.polarity_scores(df.iloc[i]['CLEAN REVIEWS'])
    compval1.append(k['compound'])
    
    i = i+1
    
compval1 = np.array(compval1)

len(compval1)

In [ ]:
df['VADER SCORE'] = compval1

In [ ]:
# converting the vader score into 3 categories 
i = 0

predicted_value = [ ] #empty series to hold our predicted values

while(i<len(df)):
    if ((df.iloc[i]['VADER SCORE'] >= 0.5)):
        predicted_value.append('positive')
        i = i+1
    elif ((df.iloc[i]['VADER SCORE'] > 0) & (df.iloc[i]['VADER SCORE'] < 0.5)):
        predicted_value.append('neutral')
        i = i+1
    elif ((df.iloc[i]['VADER SCORE'] <= 0)):
        predicted_value.append('negative')
        i = i+1
        
df['VADER SENTIMENT'] = predicted_value

In [ ]:
df.head(10)

# ML TECHINQUES

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
#splitting the data

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['CLEAN REVIEWS'], df['POSITIVELY RATED'], train_size=0.7,test_size=0.3,random_state=1)

print('Training dataset first entry:\n', X_train[0])
print("\nShape of Training Dataset", X_train.shape)

In [ ]:
vect=CountVectorizer().fit(X_train)
X_train_vectorized=vect.transform(X_train)
X_train_vectorized

# SVM

In [ ]:
from sklearn import svm

model= Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', svm.SVC(C=1.0, kernel='linear', gamma='auto'))])


model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))


# SDC

In [ ]:
# SGD Classifier
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)


print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

# MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model= Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
            ('mnb', MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None))])


model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

# PLOTS

In [ ]:
# importing the seaborn library
import seaborn as sns
 
# importings done to avoid warnings
from warnings import filterwarnings

In [ ]:
#plotting to know the spread of ratings
sns.set_style('darkgrid')
sns.countplot(x ='RATINGS', data = df)

In [ ]:
# plotting to compare number of positive, netural and negative reviews according to 'VADER'
sns.countplot(x ='VADER SENTIMENT', data = df)

In [ ]:
#plotting to compare number of positive vs number of negative reviews according to 'RATINGS'
sns.countplot(x ='POSITIVELY RATED', data = df)

In [ ]:
#plotting box plot to compare rating vs vader score
import matplotlib.pyplot as plt
plt.figure(figsize =(12, 8))
sns.boxplot(x ='RATINGS', y ='VADER SCORE', data = df)